<a href="https://colab.research.google.com/github/zeekx/build_dl_framework/blob/dev/steps/step16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backward: vector, x and itself

x = Variable(np.array(1.0))
y = add(x, x)
y.backward()
print("y=x+x=2x, f'(1)=2, but x.grad=", x.grad)



In [ ]:
import numpy as np
import heapq

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
          if not isinstance(data, np.ndarray):
            raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def reset_grad(self):
        self.grad = None

# a -> f(a) -> b
# Variable: b.creator -> f
# b.creator -> f -> f.input -> a

    def backward(self):
        def pop_func(alist):
            return heapq.heappop(alist)
        
        def append_func(alist, f):
            heapq.heappush(alist, f)

        
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        fs = [self.creator]            
        while fs:
            f = pop_func(fs)
            ygs = [o.grad for o in f.outputs]
            xgs = f.backward(*ygs)
            if not isinstance(xgs, tuple):
                xgs = (xgs,)
    
            for x, g in zip(f.inputs, xgs):
                if x.grad is None:
                    x.grad = g
                    if x.creator is not None: # append the function once when the var: x first appeared
                        append_func(fs, x.creator)
                else: # Var:x, is repeated
                    x.grad = x.grad + g
                


In [ ]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [ ]:
class Function:
    def __init__(self) -> None:
        self.generation = 0
    
    def __lt__(self, other):
            return -self.generation < -other.generation
    
    def __call__(self, *inputs): # input:[x0, x1, ...]@Variable -> [y0, y1, ...]@Variable
        self.inputs = inputs
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #unwrap
        if not isinstance(ys, tuple):
            ys = (ys,)
        
        self.generation = max([x.generation for x in self.inputs])
        self.outputs = [Variable(as_array(y)) for y in ys]
        for output in self.outputs:
            output.set_creator(self)

        return self.outputs if len(self.outputs) > 1 else self.outputs[0]

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


In [ ]:

class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx


In [ ]:
class Add(Function):
    def forward(self, a, b):
        y = a + b
        return (y,)
    
    def backward(self, gy):
        return gy, gy

In [ ]:
class Identical(Function):
    def forward(self, x):
        return x
    
    def backward(self, gy):
        return gy # ??? 1 or gy

In [ ]:
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

def add(x0, x1):
  return Add()(x0, x1)

def identical(x):
  return Identical()(x)

In [ ]:
generations = [2, 0, 1, 4, 21]
funcs = []

for g in generations:
    f = Function()
    f.generation = g
    funcs.append(f)
fs = sorted(funcs, key=lambda f: f.generation)
for f in fs:
    print(f.generation, end=', ')

In [ ]:
# y = 2X^4
# f'(x) = 8X^3
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a) , square(a))
y.backward()
print(y.data)
print(x.grad)

